In [1]:
import pandas as pd 
import numpy as np
import os
os.environ["OMP_NUM_THREADS"] = "4"
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
import sys
from functools import reduce
from AFSA import AFSA
from pred import predict
from sklearn.metrics import mean_absolute_error, accuracy_score, precision_recall_fscore_support, mean_squared_error, precision_score, recall_score

np.set_printoptions(threshold=sys.maxsize)

# Import movies table


In [2]:
#import movies table
item_df1 = pd.read_csv("data/u.item", sep="|",encoding="latin-1", 
                      names=["movie_id", "movie_title", "release_date", "video_release_date",
                             "imbd_url", "unknown", "action", "adventure", "animation",
                             "childrens", "comedy", "crime", "documentary", "drama", "fantasy", 
                             "film_noir", "horror", "musical", "mystery", "romance", 
                             "sci-fi", "thriller", "war", "western"])
item_df = item_df1.drop(['release_date','video_release_date','imbd_url','unknown'], axis=1)
item_df

,movie_id,movie_title,action,adventure,animation,childrens,comedy,crime,documentary,drama,fantasy,film_noir,horror,musical,mystery,romance,sci-fi,thriller,war,western
0,1,Toy Story (1995),0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


# Import rating table

In [3]:
#import rating table
rating_df1= pd.read_csv("data/u.data", sep="\t", names=["user_id", "movie_id", "rating", "timestamp"])
rating_df = rating_df1.drop(['timestamp'], axis=1)
rating_df

,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1
...,...,...,...
99995,880,476,3
99996,716,204,5
99997,276,1090,1
99998,13,225,2


# Devide dataset to training and testing sets

In [4]:
test = rating_df.sample(frac=0.2,random_state=200).reset_index(drop=True)
train = rating_df.drop(test.index).reset_index(drop=True)

# mirge rating table with movies table using train data
rating = pd.merge(train,item_df)
rating

,user_id,movie_id,rating,movie_title,action,adventure,animation,childrens,comedy,crime,...,fantasy,film_noir,horror,musical,mystery,romance,sci-fi,thriller,war,western
0,391,222,2,Star Trek: First Contact (1996),1,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,337,222,5,Star Trek: First Contact (1996),1,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,108,222,2,Star Trek: First Contact (1996),1,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,303,222,3,Star Trek: First Contact (1996),1,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,402,222,4,Star Trek: First Contact (1996),1,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,655,1554,2,Safe Passage (1994),0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
79996,655,1637,3,Girls Town (1996),0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
79997,655,1630,3,"Silence of the Palace, The (Saimt el Qusur) (1...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
79998,655,1641,3,Dadetown (1995),0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
#add relevent col to test set
test_actual = test
for index, row in test_actual.items():
    def categorise(row):  
        if row['rating'] >= 3.5:
            return '1'
        else:
            return '0'
    test_actual['relevent'] = test_actual.apply(lambda row: categorise(row), axis=1) #add column relevent (0 or 1)
    
test_actual

,user_id,movie_id,rating,relevent
0,846,684,5,1
1,13,596,3,0
2,224,243,2,0
3,655,310,3,0
4,559,385,4,1
...,...,...,...,...
19995,707,449,2,0
19996,854,225,1,0
19997,454,196,2,0
19998,262,747,4,1


# Bulid genres avarage rating table for each user

In [6]:
#bulid genres avarage rating table for each user
ratings1 = rating.drop(['movie_id','movie_title'], axis=1)
ratings2 = ratings1.replace({'action':{1:ratings1.rating},'adventure':{1:ratings1.rating},
                            'animation':{1:ratings1.rating},'childrens':{1:ratings1.rating},'comedy':{1:ratings1.rating},
                            'crime':{1:ratings1.rating},'documentary':{1:ratings1.rating},'drama':{1:ratings1.rating},
                            'fantasy':{1:ratings1.rating},'film_noir':{1:ratings1.rating},'horror':{1:ratings1.rating},
                            'musical':{1:ratings1.rating},'mystery':{1:ratings1.rating},'romance':{1:ratings1.rating},
                            'sci-fi':{1:ratings1.rating},'thriller':{1:ratings1.rating},'war':{1:ratings1.rating},  
                            'western':{1:ratings1.rating}})                     
ratings3 = ratings2.sort_values('user_id').drop(['rating'], axis=1)

#replace 0 with NoN 
ratings4 = ratings3.replace({'action':{0:np.nan},'adventure':{0:np.nan},
                            'animation':{0:np.nan},'childrens':{0:np.nan},'comedy':{0:np.nan},
                            'crime':{0:np.nan},'documentary':{0:np.nan},'drama':{0:np.nan},
                            'fantasy':{0:np.nan},'film_noir':{0:np.nan},'horror':{0:np.nan},
                            'musical':{0:np.nan},'mystery':{0:np.nan},'romance':{0:np.nan},
                            'sci-fi':{0:np.nan},'thriller':{0:np.nan},'war':{0:np.nan},  
                            'western':{0:np.nan}})

#get the avarage rating of each genre
ratings_av = ratings4.groupby('user_id').mean().fillna(0)

ratings_av

,action,adventure,animation,childrens,comedy,crime,documentary,drama,fantasy,film_noir,horror,musical,mystery,romance,sci-fi,thriller,war,western
user_id,,,,,,,,,,,,,,,,,,
1,3.275862,2.625000,3.571429,2.285714,3.411765,3.363636,4.8,3.912281,3.0,5.0,3.250000,2.600000,4.666667,4.181818,4.157895,3.708333,3.583333,4.000000
2,4.250000,4.000000,4.000000,4.000000,3.666667,4.000000,0.0,3.956522,0.0,5.0,3.000000,3.000000,4.000000,4.076923,3.000000,4.125000,3.000000,0.000000
3,2.833333,3.333333,0.000000,0.000000,3.333333,3.000000,5.0,3.250000,0.0,2.0,1.500000,2.000000,3.000000,3.000000,2.833333,2.444444,3.000000,0.000000
4,3.666667,3.000000,0.000000,0.000000,5.000000,4.666667,5.0,5.000000,0.0,0.0,0.000000,5.000000,4.333333,3.000000,3.666667,4.000000,4.000000,0.000000
5,3.322581,3.450000,4.000000,2.538462,3.175000,4.250000,0.0,2.428571,0.0,0.0,2.600000,3.250000,3.500000,2.555556,3.571429,3.000000,4.500000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,4.055556,3.500000,4.000000,4.000000,4.533333,4.333333,0.0,4.555556,4.0,0.0,3.000000,4.000000,4.000000,4.800000,4.125000,4.083333,5.000000,0.000000
940,3.434783,3.000000,4.500000,4.200000,3.523810,4.285714,0.0,3.604167,0.0,4.0,3.000000,3.142857,4.333333,3.541667,3.052632,3.350000,3.266667,0.000000
941,3.800000,3.857143,4.666667,4.500000,4.142857,3.000000,0.0,4.200000,0.0,0.0,0.000000,4.000000,5.000000,5.000000,3.875000,4.000000,5.000000,0.000000


# Call Artificial Fish Swarm Algorithm (AFSA) to determin the best intial centroids

In [7]:
# add AFSA_cluster

cluster_size = 40
#call AFSA
def func(x):
    my_centroids = x
    kmeans = KMeans(n_clusters=cluster_size, init=my_centroids, n_init=1)
    predictions = kmeans.fit(ratings_av)
    return predictions.inertia_

afsa = AFSA(func, n_dim=18, size_pop=8, cluster_size=cluster_size, max_iter=100,
            max_try_num=50, step=0.5, visual=0.2, q=1, delta=0.5)

my_centroids = afsa.run()

# Use the determined centroids with Kmeans Algorithm

In [9]:
#ready to cluster
kmeans_1 = KMeans(n_clusters=cluster_size, init=my_centroids, n_init=1)
predictions = kmeans_1.fit(ratings_av)
print(predictions.inertia_)

11096.866538809141


In [10]:
#add col cluster to ratings_av table
ratings_av['cluster'] = predictions.labels_

#add col cluster to rating table
user_id_cluster = ratings_av['cluster'].reset_index()
clusters_rate = pd.merge(train,user_id_cluster)

clusters_rate

,user_id,movie_id,rating,cluster
0,391,222,2,17
1,391,678,2,17
2,391,510,5,17
3,391,89,3,17
4,391,200,5,17
...,...,...,...,...
79995,941,919,5,11
79996,941,273,3,11
79997,941,1,5,11
79998,941,294,4,11


# Prediction

In [11]:
#fill (rating, relevent) prediction dataframe
test_pred = predict(clusters_rate, test_actual)

In [12]:
actual_rating = test_actual['rating'].values
pred_rating = test_pred['rating'].values

actual_relevent = test_actual['relevent'].values
pred_relevent = test_pred['relevent'].values

# Evaluation

In [13]:
mae = mean_absolute_error(actual_rating, pred_rating)
rmse = mean_squared_error(actual_rating, pred_rating, squared=False) #If True returns MSE value, if False returns RMSE value.
accuracy = accuracy_score(actual_relevent, pred_relevent)
precision = precision_score(actual_relevent, pred_relevent, average='binary', pos_label="1")
recall = recall_score(actual_relevent, pred_relevent, average='binary', pos_label="1")

In [16]:
mae, rmse

In [17]:
accuracy, precision, recall